<a href="https://colab.research.google.com/github/SteenJennings/Neural-Net-Options/blob/master/Kevin/ROTS_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# AlphaVantage CSV Processing

In [ ]:
pip install alpha_vantage

Using several API keys to pull the technical data

In [ ]:
from alpha_vantage.timeseries import  TimeSeries
from alpha_vantage.techindicators import TechIndicators
import pandas as pd
import time

tickers = ['AMD', 'SPY', 'TSLA', 'AAPL', 'AMZN', 'OPEN', 'SNAP', 'GOOG', 'NVDA', 'FSLY', 'ROKU', 'CHWY', 'TQQQ']
#ticker = tickers[8]
ticker = 'ZNGA'

#update the ti for API call limit
API_key = '04ZKSL2N50GIBN85'    #steens key
ti = TechIndicators(key = API_key, output_format='pandas')

# RSI/EMA data
data_rsi15 = ti.get_rsi(ticker, interval='daily',time_period=15, series_type='close')
data_ema5 = ti.get_ema(ticker,interval='daily',time_period=5, series_type='close')
data_ema10 = ti.get_ema(ticker,interval='daily',time_period=10, series_type='close')
data_ema20 = ti.get_ema(ticker,interval='daily',time_period=20, series_type='close')
data_ema125 = ti.get_ema(ticker,interval='daily',time_period=125, series_type='close')

#update the ti for API call limit
API_key = 'D6XW69E9UXBCXJBK'     #steen@yopmail.com
ti = TechIndicators(key = API_key, output_format='pandas')

# ADX data
data_adx5 = ti.get_adx(ticker,interval='daily',time_period=5)
data_adx10 = ti.get_adx(ticker,interval='daily',time_period=10)
data_adx20 = ti.get_adx(ticker,interval='daily',time_period=15)

#update the ti for API call limit
API_key = '50F3VWQLVV9C4HGT'    #kevins key
ti = TechIndicators(key = API_key, output_format='pandas')

# Bolinger Bands data
data_bb5 = ti.get_bbands(ticker,interval='daily',time_period=5, series_type='close')
data_bb10 = ti.get_bbands(ticker,interval='daily',time_period=10, series_type='close')
data_bb20 = ti.get_bbands(ticker,interval='daily',time_period=20, series_type='close')

# grabs the stock price data
API_key = "PK3E26W0JBG0A8LI"  # steen other email key
ts = TimeSeries(key = API_key, output_format='pandas')

# make sure that the call limit has not been met
data = ts.get_daily_adjusted(ticker, outputsize='full')
# inverts the DF to fit technical format
data = data[0].iloc[::-1]
# trims to match size of other df
data = data.iloc[149:]
# this commented section is used to trim the data after market close when the technical data is not updated but time series data is updated (1-4pm or so)
# data = data[:-1]
# data
pct_price = data[['5. adjusted close']]
pct_price = pct_price.pct_change()

Trimming the data and adding to a single dataframe

In [ ]:
# this code trims the data for each indicator type to fit into one dataframe, with ema5 heading the list
ema5_df = data_ema5[0]
ema5_df = ema5_df.iloc[145:]
ema10_df = data_ema10[0]
ema10_df = ema10_df.iloc[140:]
ema20_df = data_ema20[0]
ema20_df = ema20_df.iloc[130:]
ema125_df = data_ema125[0]
ema125_df = ema125_df.iloc[25:]
rsi15_df = data_rsi15[0]
rsi15_df = rsi15_df.iloc[134:]
adx5_df = data_adx5[0]
adx5_df = adx5_df.iloc[140:]
adx10_df = data_adx10[0]
adx10_df = adx10_df.iloc[130:]
adx20_df = data_adx20[0]
adx20_df = adx20_df.iloc[120:]
# converting the data type of the bb DF's and flipping the orientation
bb5_df = data_bb5[0].iloc[::-1]
bb10_df = data_bb10[0].iloc[::-1]
bb20_df = data_bb20[0].iloc[::-1]
# resizing the data
bb5_df = bb5_df.iloc[145:]
bb10_df = bb10_df.iloc[140:]
bb20_df = bb20_df.iloc[130:]
print(bb20_df)

#this is used to dynamically merge all dataframes using it as an element of the below loop
df_list = []

# storing everything in a tuple so we can access the column name and the DF itself
ema5_df = (ema5_df,"EMA_5")
ema10_df = (ema10_df,"EMA_10")
ema20_df = (ema20_df,"EMA_20")
ema125_df = (ema125_df,"EMA_125")
adx5_df = (adx5_df,"ADX_5")
adx10_df = (adx10_df,"ADX_10")
adx20_df = (adx20_df,"ADX_20")
bb5_df = (bb5_df,"BB_5")
bb10_df = (bb10_df,"BB_10")
bb20_df = (bb20_df,"BB_20")
rsi15_df = (rsi15_df,"RSI_15")

#appending the items to our list
df_list.append(ema5_df)
df_list.append(ema10_df)
df_list.append(ema20_df)
df_list.append(ema125_df)
df_list.append(adx5_df)
df_list.append(adx10_df)
df_list.append(adx20_df)
df_list.append(bb5_df)
df_list.append(bb10_df)
df_list.append(bb20_df)
df_list.append(rsi15_df)

#iterating across our list to fill our DF with our trimmed values
for dataframe_tuple in df_list:
  cur_dataframe, indicator_type = dataframe_tuple
  if indicator_type[0:3] == "EMA":
    temp = "EMA"
  elif indicator_type[0:3] == "RSI":
    temp = "RSI"
  elif indicator_type[0:3] == "ADX":  
    temp = "ADX"
  # this indicator has three elements. Might connect these.
  elif indicator_type[0:2] == "BB":  
    temp_list = list(cur_dataframe["Real Upper Band"])
    data[indicator_type + ' ' + 'Upper Band'] = temp_list
    continue
  else:
    temp = indicator_type
  temp_list = list(cur_dataframe[temp])
  data[indicator_type] = temp_list

data


            Real Middle Band  Real Lower Band  Real Upper Band
date                                                          
2014-12-23           24.0210          22.4639          25.5781
2014-12-24           24.0620          22.4883          25.6357
2014-12-26           24.1010          22.5185          25.6835
2014-12-29           24.1290          22.5690          25.6890
2014-12-30           24.1930          22.9125          25.4735
...                      ...              ...              ...
2021-05-24           73.5250          66.3158          80.7342
2021-05-25           73.2425          66.3953          80.0897
2021-05-26           72.9495          66.6429          79.2561
2021-05-27           72.6995          66.8036          78.5954
2021-05-28           72.5285          66.9960          78.0610

[1619 rows x 3 columns]


,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,EMA_5,EMA_10,EMA_20,EMA_125,ADX_5,ADX_10,ADX_20,BB_5 Upper Band,BB_10 Upper Band,BB_20 Upper Band,RSI_15
date,,,,,,,,,,,,,,,,,,,
2014-12-23,24.82,24.95,24.39,24.73,24.73,4739064.0,0.0,1.0,24.4830,24.2981,24.2858,26.1884,27.6571,13.7912,10.9212,25.3797,25.4728,25.5781,51.9937
2014-12-24,24.71,24.95,24.49,24.61,24.61,1220025.0,0.0,1.0,24.5254,24.3548,24.3167,26.1633,29.2847,14.2956,11.0781,25.0312,25.4368,25.6357,51.2427
2014-12-26,24.50,24.63,24.18,24.56,24.56,1405623.0,0.0,1.0,24.5369,24.3921,24.3399,26.1379,25.3411,13.9206,10.9063,24.9303,25.3759,25.6835,50.9144
2014-12-29,23.93,24.18,23.53,24.08,24.08,4095399.0,0.0,1.0,24.3846,24.3354,24.3151,26.1052,25.7245,13.0194,10.2381,25.1245,25.3822,25.6890,47.7665
2014-12-30,23.51,24.00,23.13,23.25,23.25,7023727.0,0.0,1.0,24.0064,24.1380,24.2137,26.0599,29.0198,13.0380,9.9709,25.3359,25.3841,25.4735,42.8574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-24,72.25,72.75,71.58,71.66,71.66,8579512.0,0.0,1.0,71.6816,71.7344,73.1414,80.8527,31.4082,34.1646,30.8441,74.5424,74.8618,80.7342,42.4481
2021-05-25,73.06,73.60,72.26,72.81,72.81,9398954.0,0.0,1.0,72.0577,71.9299,73.1099,80.7251,26.8930,31.9131,30.0114,74.4523,74.8710,80.0897,45.6325
2021-05-26,73.55,73.72,72.56,73.20,73.20,6598702.0,0.0,1.0,72.4385,72.1609,73.1185,80.6056,23.6664,29.7815,29.1863,74.1714,75.3378,79.2561,46.7040


Removing Columns and Making the Columns relationally based on price

In [ ]:
# modifying the data to percentages as a product of price
data['EMA_5'] = ((data['EMA_5'] - data['5. adjusted close']) / data['5. adjusted close'])
data['EMA_10'] = ((data['EMA_10'] - data['5. adjusted close']) / data['5. adjusted close'])
data['EMA_20'] = ((data['EMA_20'] - data['5. adjusted close']) / data['5. adjusted close'])
data['EMA_125'] = ((data['EMA_125'] - data['5. adjusted close']) / data['5. adjusted close'])
data['BB_5 Upper Band'] = ((data['BB_5 Upper Band'] - data['5. adjusted close']) / data['5. adjusted close'])
data['BB_10 Upper Band'] = ((data['BB_10 Upper Band'] - data['5. adjusted close']) / data['5. adjusted close'])
data['BB_20 Upper Band'] = ((data['BB_20 Upper Band'] - data['5. adjusted close']) / data['5. adjusted close'])

# dropping unneeded columns
data = data.drop(columns=['1. open', '2. high', '3. low', '4. close', '7. dividend amount', '6. volume', '8. split coefficient'])

# generating our price training list based on past price action
expected_list = [0 for _ in range(len(data))]
close_list = data['5. adjusted close'].tolist()
for i in range(1, len(close_list)-3):
    three_percent = close_list[i] * 1.03
    if ((close_list[i+1] >= three_percent) or 
    (close_list[i+2] >= three_percent) or (close_list[i+3] >= three_percent)):
      expected_list[i] = 1
data = data.assign(Expected = expected_list)
expected_list

# adding the percentage change column LAST, as all dependencies have been updated already
data['5. adjusted close'] = pct_price['5. adjusted close']
data

,5. adjusted close,EMA_5,EMA_10,EMA_20,EMA_125,ADX_5,ADX_10,ADX_20,BB_5 Upper Band,BB_10 Upper Band,BB_20 Upper Band,RSI_15,Expected
date,,,,,,,,,,,,,
2014-12-23,NaN,-0.009988,-0.017465,-0.017962,0.058973,27.6571,13.7912,10.9212,0.026272,0.030036,0.034294,51.9937,0
2014-12-24,-0.004852,-0.003438,-0.010370,-0.011918,0.063117,29.2847,14.2956,11.0781,0.017115,0.033596,0.041678,51.2427,0
2014-12-26,-0.002032,-0.000941,-0.006836,-0.008962,0.064247,25.3411,13.9206,10.9063,0.015077,0.033221,0.045745,50.9144,0
2014-12-29,-0.019544,0.012650,0.010606,0.009763,0.084103,25.7245,13.0194,10.2381,0.043376,0.054078,0.066819,47.7665,0
2014-12-30,-0.034468,0.032533,0.038194,0.041449,0.120856,29.0198,13.0380,9.9709,0.089716,0.091789,0.095634,42.8574,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-24,-0.014983,0.000301,0.001038,0.020673,0.128282,31.4082,34.1646,30.8441,0.040223,0.044680,0.126629,42.4481,0
2021-05-25,0.016048,-0.010332,-0.012088,0.004119,0.108709,26.8930,31.9131,30.0114,0.022556,0.028307,0.099982,45.6325,0
2021-05-26,0.005356,-0.010403,-0.014195,-0.001113,0.101169,23.6664,29.7815,29.1863,0.013270,0.029205,0.082734,46.7040,0


In [ ]:
import datetime 

if len(data) > (252*3):
  tod = datetime.datetime.now()
  d = datetime.timedelta(days = 365*3)
  a = tod - d
  identifier = str(a)[:10]
  data = data[(data.index >= identifier)]

data


,5. adjusted close,EMA_5,EMA_10,EMA_20,EMA_125,ADX_5,ADX_10,ADX_20,BB_5 Upper Band,BB_10 Upper Band,BB_20 Upper Band,RSI_15,Expected
date,,,,,,,,,,,,,
2018-06-04,0.057357,-0.037817,-0.042062,-0.036861,0.051874,36.6309,25.7088,29.4067,0.005280,-0.005655,0.001381,55.8258,1
2018-06-05,0.018606,-0.036928,-0.048732,-0.049267,0.032143,39.8186,25.8318,28.1864,0.020970,-0.002385,-0.015675,59.2023,1
2018-06-06,0.039362,-0.048936,-0.069347,-0.077151,-0.006837,44.9184,27.2066,27.7905,0.027577,-0.007106,-0.031715,65.3221,0
2018-06-07,-0.022277,-0.018177,-0.039387,-0.050780,0.015542,46.6313,27.8541,27.1773,0.051096,0.030101,0.000408,59.6799,1
2018-06-08,0.010633,-0.019003,-0.040496,-0.054980,0.004782,48.0017,28.4368,26.6049,0.025636,0.034306,0.001455,61.3491,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-24,-0.014983,0.000301,0.001038,0.020673,0.128282,31.4082,34.1646,30.8441,0.040223,0.044680,0.126629,42.4481,0
2021-05-25,0.016048,-0.010332,-0.012088,0.004119,0.108709,26.8930,31.9131,30.0114,0.022556,0.028307,0.099982,45.6325,0
2021-05-26,0.005356,-0.010403,-0.014195,-0.001113,0.101169,23.6664,29.7815,29.1863,0.013270,0.029205,0.082734,46.7040,0


Applying a MinMax scaler to normalize all of the data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

print(data)

data[["5. adjusted close",	"EMA_5",	"EMA_10",	"EMA_20",	"EMA_125", "ADX_5",	"ADX_10",	"ADX_20",	"BB_5 Upper Band",	"BB_10 Upper Band",	"BB_20 Upper Band",	"RSI_15"]] = min_max_scaler.fit_transform(data[["5. adjusted close",	"EMA_5",	"EMA_10",	"EMA_20",	"EMA_125", "ADX_5",	"ADX_10",	"ADX_20",	"BB_5 Upper Band",	"BB_10 Upper Band",	"BB_20 Upper Band",	"RSI_15"]])
data

            5. adjusted close     EMA_5  ...   RSI_15  Expected
date                                     ...                   
2018-06-04           0.057357 -0.037817  ...  55.8258         1
2018-06-05           0.018606 -0.036928  ...  59.2023         1
2018-06-06           0.039362 -0.048936  ...  65.3221         0
2018-06-07          -0.022277 -0.018177  ...  59.6799         1
2018-06-08           0.010633 -0.019003  ...  61.3491         1
...                       ...       ...  ...      ...       ...
2021-05-24          -0.014983  0.000301  ...  42.4481         0
2021-05-25           0.016048 -0.010332  ...  45.6325         0
2021-05-26           0.005356 -0.010403  ...  46.7040         0
2021-05-27          -0.005055 -0.003584  ...  45.7867         0
2021-05-28           0.015241 -0.012361  ...  49.0059         0

[753 rows x 13 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


,5. adjusted close,EMA_5,EMA_10,EMA_20,EMA_125,ADX_5,ADX_10,ADX_20,BB_5 Upper Band,BB_10 Upper Band,BB_20 Upper Band,RSI_15,Expected
date,,,,,,,,,,,,,
2018-06-04,0.703812,0.231681,0.286967,0.326589,0.435223,0.358777,0.287906,0.499471,0.014280,0.046723,0.135937,0.569762,1
2018-06-05,0.543308,0.236122,0.263932,0.294197,0.411264,0.406113,0.290389,0.467116,0.066071,0.056474,0.099268,0.624110,1
2018-06-06,0.629279,0.176129,0.192731,0.221393,0.363930,0.481844,0.318149,0.456619,0.087880,0.042396,0.064784,0.722616,0
2018-06-07,0.373972,0.329809,0.296206,0.290247,0.391105,0.507280,0.331223,0.440361,0.165517,0.153335,0.133844,0.631798,1
2018-06-08,0.510284,0.325683,0.292377,0.279280,0.378039,0.527630,0.342988,0.425184,0.081474,0.165872,0.136097,0.658666,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-24,0.404185,0.422135,0.435829,0.476806,0.528007,0.281221,0.458641,0.537582,0.129626,0.196804,0.405202,0.354432,0
2021-05-25,0.532713,0.369005,0.390495,0.433585,0.504239,0.214172,0.413180,0.515504,0.071306,0.147984,0.347916,0.405688,0
2021-05-26,0.488429,0.368652,0.383215,0.419923,0.495083,0.166258,0.370140,0.493627,0.040655,0.150662,0.310834,0.422935,0


Saving as a CSV based on the name of the ticker passed.

In [ ]:
filename = "%s.csv" % ticker
data.to_csv(filename, index=False)

In [ ]:
# dropping unneeded columns

dates = data.index
data.reset_index(inplace=True)
data

,date,5. adjusted close,EMA_5,EMA_10,EMA_20,EMA_125,ADX_5,ADX_10,ADX_20,BB_5 Upper Band,BB_10 Upper Band,BB_20 Upper Band,RSI_15,Expected
0,2018-06-04,0.703812,0.231681,0.286967,0.326589,0.435223,0.358777,0.287906,0.499471,0.014280,0.046723,0.135937,0.569762,1
1,2018-06-05,0.543308,0.236122,0.263932,0.294197,0.411264,0.406113,0.290389,0.467116,0.066071,0.056474,0.099268,0.624110,1
2,2018-06-06,0.629279,0.176129,0.192731,0.221393,0.363930,0.481844,0.318149,0.456619,0.087880,0.042396,0.064784,0.722616,0
3,2018-06-07,0.373972,0.329809,0.296206,0.290247,0.391105,0.507280,0.331223,0.440361,0.165517,0.153335,0.133844,0.631798,1
4,2018-06-08,0.510284,0.325683,0.292377,0.279280,0.378039,0.527630,0.342988,0.425184,0.081474,0.165872,0.136097,0.658666,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,2021-05-24,0.404185,0.422135,0.435829,0.476806,0.528007,0.281221,0.458641,0.537582,0.129626,0.196804,0.405202,0.354432,0
749,2021-05-25,0.532713,0.369005,0.390495,0.433585,0.504239,0.214172,0.413180,0.515504,0.071306,0.147984,0.347916,0.405688,0
750,2021-05-26,0.488429,0.368652,0.383215,0.419923,0.495083,0.166258,0.370140,0.493627,0.040655,0.150662,0.310834,0.422935,0
751,2021-05-27,0.445307,0.402724,0.406279,0.432187,0.499819,0.096057,0.338798,0.476979,0.035806,0.164406,0.303156,0.408170,0


# Keras NN w/Predictions

[Tutorial](https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/)

In [ ]:
#predict
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

#load the dataset
dataset = loadtxt(filename, delimiter=',', skiprows=2)
#split into input(x) and output(y) variables
X=dataset[:,0:12]
y=dataset[:,12]

#define the keras model
model = Sequential()
model.add(Dense(12, input_dim=12, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#fit the keras model on the dataset
model.fit(X, y, epochs=300, batch_size=10)

# make class predictions with the model
predictions = model.predict_classes(X)
print(predictions)

#summarize teh first X cases
for i in range(len(X)):
  print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))


Epoch 1/300
76/76 [==============================] - 1s 1ms/step - loss: 0.6713 - accuracy: 0.6418
Epoch 2/300
76/76 [==============================] - 0s 1ms/step - loss: 0.6466 - accuracy: 0.6585
Epoch 3/300
76/76 [==============================] - 0s 1ms/step - loss: 0.6530 - accuracy: 0.6432
Epoch 4/300
76/76 [==============================] - 0s 1ms/step - loss: 0.6593 - accuracy: 0.6255
Epoch 5/300
76/76 [==============================] - 0s 1ms/step - loss: 0.6514 - accuracy: 0.6332
Epoch 6/300
76/76 [==============================] - 0s 1ms/step - loss: 0.6272 - accuracy: 0.6727
Epoch 7/300
76/76 [==============================] - 0s 2ms/step - loss: 0.6592 - accuracy: 0.6152
Epoch 8/300
76/76 [==============================] - 0s 1ms/step - loss: 0.6210 - accuracy: 0.6864
Epoch 9/300
76/76 [==============================] - 0s 1ms/step - loss: 0.6489 - accuracy: 0.6366
Epoch 10/300
76/76 [==============================] - 0s 1ms/step - loss: 0.6447 - accuracy: 0.6406
Epoch 11/

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]


In [ ]:
#Change Timezone

!rm /etc/localtime
!ln -s /usr/share/zoneinfo/US/Pacific /etc/localtime
!date


Mon May 31 19:19:30 PDT 2021


In [ ]:
#export the predictions to CSV
import pandas as pd
from datetime import date
import numpy

# test_DF = pd.DataFrame(predictions, columns=['prediction'])
# test_DF['date']= data['date']
# test_DF['ticker']=ticker
# test_DF = test_DF.set_index('date')
# print(test_DF)

# # generating our price prediction list
# # this was a test to see backtest results without consecutive buy signals
# prediction_list = [0 for _ in range(len(test_DF))]
# pred_list = test_DF['prediction'].tolist()
# print(pred_list)
# for i in range(1, len(pred_list)):
#     if i == 1 and ((pred_list[i-1] == 1)):
#       pred_list[i] = 0
#     if i == 2 and ((pred_list[i-2] == 1)):
#       pred_list[i] = 0
#     if ((pred_list[i-1] == 1) or 
#     (pred_list[i-2] == 1) or (pred_list[i-3] == 1)):
#       pred_list[i] = 0

# test_DF['prediction']= pred_list
# print(test_DF)

test_DF = pd.DataFrame(predictions, columns=['prediction'])
test_DF['date']= data['date']
test_DF['expected'] = data["Expected"].copy()
comparison_column = numpy.where(test_DF['prediction'] == test_DF['expected'], 1, 0)
test_DF["Equal"] = comparison_column
correctBuySignal = numpy.where((test_DF['prediction'] == test_DF['expected']) & (test_DF['Equal'] == test_DF['expected']), 1, 0)
test_DF["correctBuySignal"] = correctBuySignal
test_DF['ticker'] = ticker
test_DF = test_DF.set_index('date')
print(test_DF)

today = date.today()
saveLocation = "/content/%s_pred_%s.csv" %(ticker,today)
test_DF.to_csv(saveLocation)
#test_DF


            prediction  expected  Equal  correctBuySignal ticker
date                                                            
2018-06-04           1         1      1                 1     JD
2018-06-05           0         1      0                 0     JD
2018-06-06           1         0      0                 0     JD
2018-06-07           1         1      1                 1     JD
2018-06-08           0         1      0                 0     JD
...                ...       ...    ...               ...    ...
2021-05-21           0         0      1                 0     JD
2021-05-24           0         0      1                 0     JD
2021-05-25           0         0      1                 0     JD
2021-05-26           0         0      1                 0     JD
2021-05-27           0         0      1                 0     JD

[752 rows x 5 columns]
